from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
from urllib.parse import urljoin

In [ ]:
# Launch browser
def launch_driver(headless=False):
    chrome_options = Options()
    chrome_options.binary_location = '/usr/bin/google-chrome'
    if headless:
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

driver = launch_driver()

try:
    url = 'https://science.nasa.gov/mars/stories/'
    driver.get(url)

    # Wait for page to load at least one article
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'hds-content-item-heading'))
    )

    html = driver.page_source
    soup = bs(html, 'html.parser')

    # Find all article links
    articles = soup.find_all('a', class_='hds-content-item-heading')

    for article in articles:
        title_tag = article.find('div', class_='hds-a11y-heading-22')
        href = article.get('href')

        if title_tag and href:
            title = title_tag.get_text(strip=True)
            print(f"Title: {title}")
            print(f"Link: {href}\n")

except Exception as e:
    print("Error:", e)

finally:
    driver.quit()

Title: Advances in NASA Imaging Changed How World Sees Mars
Paragraph: Sixty years ago, NASA’s Mariner 4 captured groundbreaking views of the Red Planet, leading to a steady stream of advances in the cameras used to study other worlds. In 1965, NASA’s Mariner 4 mission brought Mars into American living rooms,…

Title: NASA Mars Orbiter Learns New Moves After Nearly 20 Years in Space
Paragraph: The Mars Reconnaissance Orbiter is testing a series of large spacecraft rolls that will help it hunt for water. After nearly 20 years of operations, NASA’s Mars Reconnaissance Orbiter (MRO) is on a roll, performing a new maneuver to squeeze…

Title: NASA’s Perseverance Rover Scours Mars for Science
Paragraph: In addition to drilling rock core samples, the science team has been grinding its way into rocks to make sense of the scientific evidence hiding just below the surface. On June 3, NASA’s Perseverance Mars rover ground down a portion…

Title: NASA’s Curiosity Mars Rover Starts Unpacking Boxwo

## JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image at https://www.jpl.nasa.gov/images/?search=&category=Mars.
* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called 'featured_image_url'.
* Make sure to find the image url to the full size .jpg image.
* Make sure to save a complete url string for this image.

In [18]:
driver = launch_driver()

try:
    # Navigate to JPL Mars Images
    url = "https://www.jpl.nasa.gov/images?search=&category=Mars"
    driver.get(url)

    # Wait for image to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "BaseImage"))
    )

    # Parse HTML
    html = driver.page_source
    soup = bs(html, "html.parser")

    # Find featured image
    image_tag = soup.find("img", class_="BaseImage")
    featured_image_url = image_tag["src"]

    print("Featured Mars Image URL:", featured_image_url)

except Exception as e:
    print("Error:", e)

finally:
    driver.quit()

Featured Mars Image URL: https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA26580.2e16d0ba.fill-400x400-c50.jpg


## Mars Weather
* Visit the Curiosity Rover Updates at https://science.nasa.gov/mission/msl-curiosity/science-updates/
* Scrape the latest news. 
* Save the tweet text for the weather report as a variable called 'curiosity_updates'.

In [19]:
driver = launch_driver()

try:
    url = "https://science.nasa.gov/mission/msl-curiosity/science-updates/"
    driver.get(url)

    # Wait for blog content to appear
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "hds-content-item-heading"))
    )

    # Parse with BeautifulSoup
    html = driver.page_source
    soup = bs(html, "html.parser")

    # Get the first blog entry
    first_entry = soup.find("a", class_="hds-content-item-heading")
    title = first_entry.find("div", class_="hds-a11y-heading-22").get_text(strip=True)
    link = first_entry["href"]

    description_tag = first_entry.find_next("p", class_="margin-top-0 margin-bottom-1")
    description = description_tag.get_text(strip=True) if description_tag else ""

    date_tag = first_entry.find_next("div", class_="label margin-y-1")
    post_date = date_tag.get_text(strip=True) if date_tag else ""

    # Combine into a single string
    curiosity_updates = f"{title} ({post_date})\n{description}\nLink: {link}"
    print(curiosity_updates)

except Exception as e:
    print("Error while scraping Curiosity updates:", e)

finally:
    driver.quit()

Curiosity Blog, Sols 4593-4594: Three Layers and a Lot of Structure at Volcán Peña Blanca (Jul 10, 2025)
Written by Susanne P. Schwenzer, Professor of Planetary Mineralogy at The Open University, UK Earth planning date: Monday, July 7, 2025 A few planning sols ago, we spotted a small ridge in the landscape ahead of us. Ridges and structures…
Link: https://science.nasa.gov/blog/curiosity-blog-sols-4593-4594-three-layers-and-a-lot-of-structure-at-volcan-pena-blanca/


## Mars Facts
* Visit the Mars Facts webpage at https://space-facts.com/mars/
* Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Use Pandas to convert the data to a HTML table string.

In [22]:
# Target URL
facts_url = "https://space-facts.com/mars/"

# Read all tables on the page
tables = pd.read_html(facts_url)

# The main Mars facts table is usually the first one
df = tables[0]

# Clean and format the DataFrame
df.columns = ['Description', 'Mars']
df.set_index('Description', inplace=True)

# Preview cleaned table
print(df)

                                               Mars
Description                                        
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers


In [23]:
# Convert to HTML string
mars_df_html = df.to_html(classes="table table-striped", border=0)
mars_df_html = mars_df_html.replace("\n", "")

# Save to HTML file
df.to_html('mars_facts.html', classes="table table-striped", border=0)

# Optional: Save HTML string to a variable for use in dashboards
print(mars_df_html)

<table class="dataframe table table-striped">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


## Mars Map
* Visit the USGS Astrogeology site of ArcGIS Mars Interactive Map Viewer located at: https://usgs.maps.arcgis.com/apps/webappviewer/index.html?id=f693b580b2464e6989f32e4266199552
* Embed the ArcGIS Mars Interactive Map Viewer

In [ ]:
<iframe 
    src="https://usgs.maps.arcgis.com/apps/webappviewer/index.html?id=f693b580b2464e6989f32e4266199552" 
    width="100%" 
    height="800" 
    style="border:none;"
    allowfullscreen
    loading="lazy">
</iframe>


In [26]:
from IPython.display import IFrame

IFrame(
    src="https://usgs.maps.arcgis.com/apps/webappviewer/index.html?id=f693b580b2464e6989f32e4266199552",
    width='100%',
    height=800
)
